In [2]:
# import packages for pipeline
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import eli5

In [3]:
# import and merge data
train_values = pd.read_csv('data/train_values.csv')
train_labels = pd.read_csv('data/train_labels.csv')
train_data = train_values.merge(train_labels, left_on='building_id', right_on='building_id')

In [5]:
# check data types
train_data.dtypes

building_id                                int64
geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_c

In [6]:
# dropping id for model training, and doing data prep
train_data = train_data.drop('building_id', axis=1) # axis 1 is to drop a column (0 would be for rows)
numeric_features = train_data.select_dtypes(include=['int64', 'float64']).drop(['damage_grade'], axis=1).columns
categorical_features = train_data.select_dtypes(include=['object']).columns
X = train_data.drop('damage_grade', axis=1)
y = train_data['damage_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # test size does percentage split for train / test (20% in this case)

In [7]:
# pipeline that imputes any missing values, applies a standard scaler to the numerical features, converts any categorical features into numerical and then fits a classifier
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot', OneHotEncoder())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier',  LogisticRegression(class_weight='balanced', random_state=0))])
    
model = pipe.fit(X_train, y_train)

/Users/goodrules/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/goodrules/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [8]:
# inspect model quality
target_names = y_test.unique().astype(str)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           3       0.45      0.61      0.52      5002
           1       0.63      0.64      0.63     29669
           2       0.49      0.44      0.46     17450

    accuracy                           0.57     52121
   macro avg       0.52      0.56      0.54     52121
weighted avg       0.57      0.57      0.57     52121



In [9]:
# one hot encoding to add cat variables to numerical feature list
onehot_columns = list(pipe.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot'].get_feature_names(input_features=categorical_features))
numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)

In [10]:
# feature importance
eli5.explain_weights(pipe.named_steps['classifier'], top=50, feature_names=numeric_features_list)

Explanation(estimator="LogisticRegression(C=1.0, class_weight='balanced', dual=False,\n                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,\n                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',\n                   random_state=0, solver='warn', tol=0.0001, verbose=0,\n                   warm_start=False)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='plan_configuration_o', weight=0.5201418662551045, std=None, value=None), FeatureWeight(feature='plan_configuration_n', weight=0.49737830764311763, std=None, value=None), FeatureWeight(feature='other_floor_type_s', weight=0.36302971547170604, std=None, value=None), FeatureWeight(feature='position_s', weight=0.35234376386638233, std=None, value=None), FeatureWeight(feature='foundation_type_i', weight=0.27237545920876294, std=None, value=None), FeatureWeight(feature='foundation_type_h', weight=0.26090799372739587, std=None, value=None), FeatureWeight(feature='geo_level_1_id', weight=0.24172620495791156, std=None, value=None), FeatureWeight(feature='ground_floor_type_v', weight=0.16348854763611673, std=None, value=None), FeatureWeight(feature='has_secondary_use_hotel', weight=0.15726724482553064, std=None, value=None), FeatureWeight(feature='has_superstructure_timber', weight=0.15176827484437602, std=None, value=None), FeatureWeight(feature='plan_configuration_a', weight=0.14630686558869868, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_engineered', weight=0.11158572173465028, std=None, value=None), FeatureWeight(feature='legal_ownership_status_a', weight=0.1107230801527942, std=None, value=None), FeatureWeight(feature='has_secondary_use_rental', weight=0.11006035185635733, std=None, value=None), FeatureWeight(feature='plan_configuration_u', weight=0.10143467953957491, std=None, value=None), FeatureWeight(feature='roof_type_x', weight=0.08532177756072704, std=None, value=None), FeatureWeight(feature='has_secondary_use_other', weight=0.08022772761384586, std=None, value=None), FeatureWeight(feature='has_superstructure_other', weight=0.06354870708108475, std=None, value=None), FeatureWeight(feature='geo_level_2_id', weight=0.05698880090700094, std=None, value=None), FeatureWeight(feature='has_superstructure_rc_non_engineered', weight=0.056686060026913634, std=None, value=None), FeatureWeight(feature='has_secondary_use_agriculture', weight=0.05513764532848793, std=None, value=None)], neg=[FeatureWeight(feature='plan_configuration_f', weight=-1.2862271543596155, std=None, value=None), FeatureWeight(feature='foundation_type_r', weight=-0.6820829503440423, std=None, value=None), FeatureWeight(feature='position_o', weight=-0.6042719637536079, std=None, value=None), FeatureWeight(feature='legal_ownership_status_w', weight=-0.5865003755532814, std=None, value=None), FeatureWeight(feature='has_superstructure_mud_mortar_stone', weight=-0.5287301642572024, std=None, value=None), FeatureWeight(feature='roof_type_q', weight=-0.5010703805680402, std=None, value=None), FeatureWeight(feature='<BIAS>', weight=-0.500904315799293, std=None, value=None), FeatureWeight(feature='foundation_type_u', weight=-0.39260013688150097, std=None, value=None), FeatureWeight(feature='other_floor_type_q', weight=-0.335292034033443, std=None, value=None), FeatureWeight(feature='has_superstructure_adobe_mud', weight=-0.327212924476029, st